In [1]:
import nannyml as nml
from IPython.display import display

reference_df, analysis_df, _ = nml.load_synthetic_binary_classification_dataset()
analysis_df = analysis_df[analysis_df['timestamp'] == analysis_df['timestamp'].min()]
display(reference_df.head())

,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,identifier,work_home_actual,timestamp,y_pred_proba,period,y_pred
0,5.962247,40K - 60K €,2.119485,8.568058,False,Friday,0.212653,0,1,2014-05-09 22:27:20,0.99,reference,1
1,0.535872,40K - 60K €,2.357199,5.425382,True,Tuesday,4.927549,1,0,2014-05-09 22:59:32,0.07,reference,0
2,1.969519,40K - 60K €,2.366849,8.247158,False,Monday,0.520817,2,1,2014-05-09 23:48:25,1.00,reference,1
3,2.530410,20K - 40K €,2.318722,7.944251,False,Tuesday,0.453649,3,1,2014-05-10 01:12:09,0.98,reference,1
4,2.253635,60K+ €,2.221265,8.884478,True,Thursday,5.695263,4,1,2014-05-10 02:21:34,0.99,reference,1


In [2]:
print(reference_df.head().to_markdown(tablefmt="grid"))

+----+------------------------+----------------+-----------------------+------------------------------+--------------------+-----------+----------+--------------+--------------------+---------------------+----------------+-----------+----------+
|    |   distance_from_office | salary_range   |   gas_price_per_litre |   public_transportation_cost | wfh_prev_workday   | workday   |   tenure |   identifier |   work_home_actual | timestamp           |   y_pred_proba | period    |   y_pred |
+====+========================+================+=======================+==============================+====================+===========+==========+==============+====================+=====================+================+===========+==========+
|  0 |               5.96225  | 40K - 60K €    |               2.11948 |                      8.56806 | False              | Friday    | 0.212653 |            0 |                  1 | 2014-05-09 22:27:20 |           0.99 | reference |        1 |
+----+----------

In [3]:
column_names = ['distance_from_office', 'salary_range', 'gas_price_per_litre', 'public_transportation_cost', 'wfh_prev_workday', 'workday', 'tenure', 'y_pred_proba', 'y_pred']
calc = nml.UnivariateDriftCalculator(
    column_names=column_names,
    timestamp_column_name='timestamp',
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
    categorical_methods=['chi2', 'jensen_shannon'],
    chunk_period='D',
)
calc.fit(reference_df)
univariate_results = calc.calculate(analysis_df)
display(univariate_results.filter(period='analysis', column_names=['distance_from_office']).to_df(multilevel=False))

/home/niels/Code/nml/nannyml/nannyml/nannyml/chunk.py:181: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(


,chunk_index,chunk_end_date,chunk_end_index,chunk_key,chunk_period,chunk_start_date,chunk_start_index,distance_from_office_jensen_shannon_alert,distance_from_office_jensen_shannon_lower_threshold,distance_from_office_jensen_shannon_upper_threshold,distance_from_office_jensen_shannon_value,distance_from_office_kolmogorov_smirnov_alert,distance_from_office_kolmogorov_smirnov_lower_threshold,distance_from_office_kolmogorov_smirnov_upper_threshold,distance_from_office_kolmogorov_smirnov_value
0,0,2017-08-31 23:59:59.999999999,0,2017-08-31,analysis,2017-08-31,0,True,None,0.1,0.753887,False,None,None,0.84558


In [4]:
print(univariate_results.filter(period='analysis', column_names=['distance_from_office']).to_df(multilevel=False).to_markdown(tablefmt="grid"))

+----+---------------+-------------------------------+-------------------+-------------+----------------+---------------------+---------------------+---------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+---------------------------------------------+-------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------+
|    |   chunk_index | chunk_end_date                |   chunk_end_index | chunk_key   | chunk_period   | chunk_start_date    |   chunk_start_index | distance_from_office_jensen_shannon_alert   | distance_from_office_jensen_shannon_lower_threshold   |   distance_from_office_jensen_shannon_upper_threshold |   distance_from_office_jensen_shannon_value | distance_from_office_kolmogorov_smirnov_alert   | distance_from_office_k

In [5]:
calc2 = nml.DataReconstructionDriftCalculator(
    column_names=column_names,
    timestamp_column_name='timestamp',
    chunk_period='D'
)
calc2.fit(reference_df)
multivariate_results = calc2.calculate(analysis_df)
display(multivariate_results.filter(period='analysis').to_df(multilevel=False))

/home/niels/Code/nml/nannyml/nannyml/nannyml/chunk.py:181: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(


,chunk_key,chunk_index,chunk_start_index,chunk_end_index,chunk_start_date,chunk_end_date,chunk_period,reconstruction_error_sampling_error,reconstruction_error_value,reconstruction_error_upper_confidence_boundary,reconstruction_error_lower_confidence_boundary,reconstruction_error_upper_threshold,reconstruction_error_lower_threshold,reconstruction_error_alert
0,2017-08-31,0,0,0,2017-08-31,2017-08-31 23:59:59.999999999,analysis,0.570533,1.557116,3.268714,-0.154482,1.367633,1.919277,False


In [6]:
print(multivariate_results.filter(period='analysis').to_df(multilevel=False).to_markdown(tablefmt="grid"))

+----+-------------+---------------+---------------------+-------------------+---------------------+-------------------------------+----------------+---------------------------------------+------------------------------+--------------------------------------------------+--------------------------------------------------+----------------------------------------+----------------------------------------+------------------------------+
|    | chunk_key   |   chunk_index |   chunk_start_index |   chunk_end_index | chunk_start_date    | chunk_end_date                | chunk_period   |   reconstruction_error_sampling_error |   reconstruction_error_value |   reconstruction_error_upper_confidence_boundary |   reconstruction_error_lower_confidence_boundary |   reconstruction_error_upper_threshold |   reconstruction_error_lower_threshold | reconstruction_error_alert   |
+====+=============+===============+=====================+===================+=====================+============================

In [7]:
handler = nml.SlackNotificationHandler(webhook_url='https://hooks.slack.com/services/SOME/WEBHOOK/URL')

In [8]:
handler.handle(results=[univariate_results, multivariate_results])